In [13]:
import FaaSET
import pprint
import pandas as pd
import numpy as np
import time

pp = pprint.PrettyPrinter(indent=4)

In [2]:
@FaaSET.cloud_function(platform="AWS Docker Debian")
def cache_bench_test(request, context): 
    return {"message": "Dummy Function"}   

cache_bench_test({}, None)

Build: Loading config...
Build: Purging previous build...
Build: Copying files...
Build: Cleaning up...
Build: Building Docker Image...
Sending build context to Docker daemon  502.3kB

Step 1/16 : FROM python:3.8-slim-buster
 ---> c706fbd78639
Step 2/16 : RUN apt update
 ---> Using cache
 ---> 9e15b7761466
Step 3/16 : RUN apt install -y curl build-essential
 ---> Using cache
 ---> d20ecd952a4a
Step 4/16 : ENV FUNCTION_DIR="/var/task"
 ---> Using cache
 ---> 8c2971bc0243
Step 5/16 : RUN mkdir -p ${FUNCTION_DIR}
 ---> Using cache
 ---> ec6f531b3418
Step 6/16 : COPY . ${FUNCTION_DIR}
 ---> 2c9805aaf0ee
Step 7/16 : RUN pip install boto3 botocore
 ---> Running in b0a619df42c1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 KB 19.9 MB/s eta 0:00:0

 ---> dd9f08bfa3c8
Step 7/16 : RUN pip install boto3 botocore
 ---> Running in 6517fb34a0a8


KeyboardInterrupt: 

In [43]:
%%writefile functions/cache_bench_test/Dockerfile

#FROM python:buster as build-image
FROM python:3.8-slim-buster

# Update and install curl. 
RUN apt update
RUN apt install -y curl build-essential moreutils expect #stdbuf

# Define and create function directory
ENV FUNCTION_DIR="/var/task"
RUN mkdir -p ${FUNCTION_DIR}

# Copy function code
COPY . ${FUNCTION_DIR}

RUN pip install boto3 botocore

# Install the runtime interface client
RUN pip install \
        --target ${FUNCTION_DIR} \
        awslambdaric

# Install the runtime interface emulator
RUN curl -Lo /usr/local/bin/aws-lambda-rie https://github.com/aws/aws-lambda-runtime-interface-emulator/releases/latest/download/aws-lambda-rie
RUN chmod +x /usr/local/bin/aws-lambda-rie

# Set working directory
WORKDIR ${FUNCTION_DIR}

# Compile C code
# RUN gcc -o stream.10M -O -DSTREAM_ARRAY_SIZE=10000000 -DNTIMES=10 stream_small.c
RUN cd ${FUNCTION_DIR}/llcbench  && make clean && make cache-bench 


# Set entry script that determines whether to run RIE if running locally
COPY ./entry_script.sh /entry_script.sh
RUN chmod +x /entry_script.sh
ENTRYPOINT [ "/entry_script.sh" ]

# Set handler
CMD [ "lambda_function.lambda_handler" ]

Overwriting functions/cache_bench_test/Dockerfile


In [33]:
@FaaSET.cloud_function(platform="AWS Docker Debian", config={"memory": 259})
def cache_bench_test(request, context): 
    import subprocess, datetime, time

    from SAAF import Inspector
    inspector = Inspector()
    inspector.inspectAll()
    
#     command = "/var/task/llcbench/cachebench/cachebench -w -d1 -x1 -m20 -e1"

    command = "bash /var/task/run_cache_bench.sh" 
#     command = "stdbuf -i0 -o0 -e0 /var/task/llcbench/cachebench/cachebench -w -d1 -x1 -m10 -e1 | ts '[%Y-%m-%d %H:%M:%.S]'"
    start_time = float(request['start_time'])
    print("start time is: {}".format(start_time))
    
    now = datetime.datetime.now().timestamp()
    if start_time >= now:
        time.sleep(start_time - now)
    output = subprocess.check_output(command.split()).decode('ascii')

    inspector.addAttribute("cache_bench_metric", output)
    inspector.inspectAllDeltas()
    return inspector.finish()

In [34]:
import datetime
now = datetime.datetime.now().timestamp()

In [36]:
%time ret = cache_bench_test({'start_time': int(now + 3)}, None)

CPU times: user 6.65 ms, sys: 28.1 ms, total: 34.7 ms
Wall time: 4min 1s


In [7]:
from multiprocessing.pool import Pool

In [8]:
def run_function(now=None):
    if now is None:
        now = datetime.datetime.now().timestamp()
    params = {"start_time": now}
    return cache_bench_test(params, None)

In [9]:
def run_exp(con=20):
    p = Pool(con)
    now = datetime.datetime.now().timestamp()
    result = p.map(run_function, [now+5] * con)
    return result

In [10]:
MAX_CON = 30

In [ ]:
raw_data = []
# now = datetime.datetime.now().timestamp()
for i in range(1, MAX_CON + 1):
    print('current round is: {}'.format(i))
    tmp = run_exp(i)
    tmp = run_exp(i)
    raw_data.append(tmp)

current round is: 1


In [ ]:
with open('/home/itadmin/exp_data/cache_bench/6.9/rawdata.txt', 'w') as f:
    json.dump(raw_data, f)